<a href="https://colab.research.google.com/github/worldofaryavart/LLMs-from-scratch/blob/main/LLMsChap2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
OPT = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [ ]:
inp = "The quick brown fox jumps over the lazy dog"
inp_tokenized = tokenizer( inp, return_tensors="pt" )
print( inp_tokenized['input_ids'].size() )
print( inp_tokenized )

torch.Size([1, 10])
{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
print( OPT.model )

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 2048, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTAttention(
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, e

In [ ]:
embedded_input = OPT.model.decoder.embed_tokens(inp_tokenized['input_ids'])
print( "Layer:\t", OPT.model.decoder.embed_tokens)
print( "Size: \t", embedded_input.size() )
print("Output: \t", embedded_input)

Layer:	 Embedding(50272, 2048, padding_idx=1)
Size: 	 torch.Size([1, 10, 2048])
Output: 	 tensor([[[-0.0407,  0.0519,  0.0574,  ..., -0.0263, -0.0355, -0.0260],
         [-0.0371,  0.0220, -0.0096,  ...,  0.0265, -0.0166, -0.0030],
         [-0.0455, -0.0236, -0.0121,  ...,  0.0043, -0.0166,  0.0193],
         ...,
         [ 0.0007,  0.0267,  0.0257,  ...,  0.0622,  0.0421,  0.0279],
         [-0.0126,  0.0347, -0.0352,  ..., -0.0393, -0.0396, -0.0102],
         [-0.0115,  0.0319,  0.0274,  ..., -0.0472, -0.0059,  0.0341]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>)


In [ ]:
embed_pos_input = OPT.model.decoder.embed_positions(inp_tokenized['attention_mask'])
print( "Layer:\t", OPT.model.decoder.embed_positions )
print( "Size:\t", embed_pos_input.size() )
print( "Output:\t", embed_pos_input )

Layer:	 OPTLearnedPositionalEmbedding(2050, 2048)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-8.1406e-03, -2.6221e-01,  6.0768e-03,  ...,  1.7273e-02,
          -5.0621e-03, -1.6220e-02],
         [-8.0585e-05,  2.5000e-01, -1.6632e-02,  ..., -1.5419e-02,
          -1.7838e-02,  2.4948e-02],
         [-9.9411e-03, -1.4978e-01,  1.7557e-03,  ...,  3.7117e-03,
          -1.6434e-02, -9.9087e-04],
         ...,
         [ 3.6979e-04, -7.7454e-02,  1.2955e-02,  ...,  3.9330e-03,
          -1.1642e-02,  7.8506e-03],
         [-2.6779e-03, -2.2446e-02, -1.6754e-02,  ..., -1.3142e-03,
          -7.8583e-03,  2.0096e-02],
         [-8.6288e-03,  1.4233e-01, -1.9012e-02,  ..., -1.8463e-02,
          -9.8572e-03,  8.7662e-03]]], device='cuda:0', dtype=torch.float16,
       grad_fn=<EmbeddingBackward0>)


In [ ]:
embed_position_input = embedded_input + embed_pos_input
hidden_states, _, _ = OPT.model.decoder.layers[0].self_attn( embed_position_input )
print( "Layer:\t", OPT.model.decoder.layers[0].self_attn )
print( "Size:\t", hidden_states.size() )
print( "Output:\t", hidden_states )

Layer:	 OPTAttention(
  (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
  (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
  (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
  (out_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0119, -0.0110,  0.0056,  ...,  0.0094,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         ...,
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093],
         [-0.0119, -0.0110,  0.0056,  ...,  0.0095,  0.0013,  0.0093]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MatMul8bitLtBackward>)


In [1]:
from transformers import AutoModel, AutoTokenizer
BART = AutoModel.from_pretrained("facebook/bart-large")
print(BART)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

BartModel(
  (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-11): 12 x BartEncoderLayer(
        (self_attn): BartSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=

In [2]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sum = summarizer("""
  BartModel(
 (shared): Embedding(50265, 1024, padding_idx=1)
 (encoder): BartEncoder(
 (embed_tokens): Embedding(50265, 1024, padding_idx=1)
 (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
 (layers): ModuleList(
 (0-11): 12 x BartEncoderLayer(
 (self_attn): BartAttention(
 (k_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (v_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (q_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (out_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 )
 (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 (activation_fn): GELUActivation()
 (fc1): Linear(in_features=1024, out_features=4096,
bias=True)
 (fc2): Linear(in_features=4096, out_features=1024,
bias=True)
 (final_layer_norm): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 )
 )
 (layernorm_embedding): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 )
 (decoder): BartDecoder(
 (embed_tokens): Embedding(50265, 1024, padding_idx=1)
 (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
 (layers): ModuleList(
 (0-11): 12 x BartDecoderLayer(
 (self_attn): BartAttention(
 (k_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (v_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (q_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (out_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 )
 (activation_fn): GELUActivation()
 (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 (encoder_attn): BartAttention(
 (k_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (v_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (q_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 (out_proj): Linear(in_features=1024, out_features=1024,
bias=True)
 )
 (encoder_attn_layer_norm): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 (fc1): Linear(in_features=1024, out_features=4096,
bias=True)
 (fc2): Linear(in_features=4096, out_features=1024,
bias=True)
 (final_layer_norm): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 )
 )
 (layernorm_embedding): LayerNorm((1024,), eps=1e-05,
elementwise_affine=True)
 ))
""", min_length=20, max_length=50)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
print(sum[0]['summary_text'])

BartEncoder: BartEncoder (embed_tokens): Embedding(50265, 1024, padding_idx=1) BartModel: BartLearnedPositionalEmbedding (1026, 1024


In [5]:
BERT = AutoModel.from_pretrained("bert-base-uncased")
print(BERT)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [6]:
classifier = pipeline("text-classification",
                      model="nlptown/bert-base-multilingual-uncased-sentiment")
lbl = classifier("""This restaurant is awesome.""")
print(lbl)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': '5 stars', 'score': 0.8550480604171753}]


In [7]:
gpt2= AutoModel.from_pretrained("gpt2")
print(gpt2)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


In [9]:
generator = pipeline(model="gpt2")
gen = generator("Hello, I'm a language model,", max_length=300, num_return_sequences=5)
for item in gen:
  print(">", item['generated_text'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


> Hello, I'm a language model, not a language model"

"And then this little little snippet of history, what is history?" The man asked, and then leaned down slightly, his hand on the door handle.

"A history of mankind or rather a history of all that happened." I said, my voice full of pride. "History is that you take something that isn't good and make it the biggest thing that could possibly happen."

The man reached up the door and placed the keys on the desk.

"That is going to work!" I said. "What do you want to do with the key?"

"Leave the keys here!" The man said, as I grabbed both my hands.

"I'll just go and talk to the woman before I left. She's here to talk to me..."

"... I don't think I want you either." said the woman.

"Of course not. It was a woman who called her last name. She said she was the last one who would call herself, and I'll go get her. When I'm home she can call you again whenever I want, and I can always use my phone. How does she know this one girl over he